In [35]:
from pathlib import Path
import numpy as np

from aiida import load_profile
from aiida.orm import load_node, load_group, Dict

from aiida_vasp.utils.aiida_utils import get_data_class

from aiida_vasp.parsers.content_parsers.incar import IncarParser
from aiida_vasp.parsers.content_parsers.kpoints import KpointsParser
from aiida_vasp.parsers.content_parsers.potcar import MultiPotcarIo
from aiida_vasp.parsers.content_parsers.poscar import PoscarParser

from aiida_vasp.assistant.parameters import inherit_and_merge_parameters, ParametersMassage

load_profile('lauerm-prod')

Profile<uuid='3199e329b1c748319390ff8be24b9c6c' name='lauerm-prod'>

In [34]:

# later I will have to make this more general, then I can put it as a util
def create_aiida_vasp_folder(workchain_pk, out_folder, pot_mapping=None, pot_family="PBE.64"):

    def restore_incar(inputs) -> Dict:
        parameters = inherit_and_merge_parameters(inputs)
        settings = inputs.settings
        
        new_para = ParametersMassage(
            parameters,
            settings = settings
        )
        return Dict(new_para.parameters["incar"]) 


    workchain_node = load_node(workchain_pk)

    inputs = workchain_node.inputs
    outputs = workchain_node.outputs

    if isinstance(out_folder, str):
        out_folder = Path(out_folder)
    elif not isinstance(out_folder, Path):
        raise ValueError(f"out_folder is of wrong type {type(out_folder)} - must either be str or pathlib.Path")

    incar = restore_incar(inputs)
    kpoints = inputs.kpoints

    str_prec = inputs.settings.get_dict().get('poscar_prcision', 10)
    inp_structure = inputs.structure
    out_structure = outputs.relax.structure

    if pot_mapping is None:
        pot_mapping = {el.name: el.name for el in np.unique(inp_structure.get_pymatgen().species)}


    incar_parser = IncarParser(data = incar)
    incar_parser.write(out_folder / "INCAR")

    kpoints_parser = KpointsParser(data = kpoints)
    kpoints_parser.write(out_folder / 'KPOINTS')

    potcars = get_data_class('vasp.potcar').get_potcars_from_structure(structure   = inp_structure, 
                                            family_name = pot_family, 
                                            mapping     = pot_mapping
                                            )
    potcar_parser = MultiPotcarIo.from_structure(inp_structure, potcars)
    potcar_parser.write(out_folder / 'POTCAR')

    
    poscar_parser = PoscarParser(data = inp_structure, precision = str_prec)
    poscar_parser.write(out_folder / "POSCAR")

    contcar_parser = PoscarParser(data = out_structure, precision = str_prec)
    contcar_parser.write(out_folder / "CONTCAR")


In [40]:
import os 
os.getcwd()

'/home/mlauer/Documents/work/heiliger_work/aiida-kkr/aiida_vasp_production/Nitiride_Relaxation'

In [ ]:
group = load_group(4)

top_folder = Path('../test')

for idx, node in enumerate(group.nodes):

    workchain = node
    name = "".join(np.unique(node.inputs.structure.get_pymatgen().species))

    write_folder = top_folder / str(idx) 
    write_folder.mkdir(parents = True, exist_ok=True)
    
    for wc in workchain.called:
        if wc.process_label == "RelaxWorkChain":
            
            create_aiida_vasp_folder(wc.pk, write_folder)